In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

from skopt.space import Real, Integer
from skopt.utils import use_named_args
import itertools
from sklearn.metrics import log_loss
from skopt import gp_minimize

os.chdir('../..')

from src.utils_k import add_type_features
from src.utils_j import generate_validation_set, test_lgbm

pd.options.display.max_columns = 9999
pd.options.display.max_rows = 9999

In [2]:
from src.utils_f import duration_feature
import lightgbm as lgbm

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
os.getcwd()

'/home/dsgpucp/Documents/DSG-2018-Finals'

In [5]:
test_session = pd.read_csv('data/raw/random_submission.csv')
test_tracking = pd.read_csv('data/raw/test_tracking.csv')
all_session = pd.read_csv('data/raw/train_session.csv') 
all_tracking = pd.read_csv('data/raw/train_tracking.csv')
productid_category = pd.read_csv('data/raw/productid_category.csv')

In [6]:
trn_session, trn_tracking, val_session, val_tracking = generate_validation_set(all_session, all_tracking, test_size=0.4)

In [7]:
proc_train = add_type_features(trn_session[['sid']], trn_tracking)
proc_val = add_type_features(val_session[['sid']], val_tracking)
proc_all = add_type_features(all_session[['sid']], all_tracking)
proc_test = add_type_features(test_session, test_tracking)

In [8]:
proc_train = duration_feature(proc_train, trn_tracking)
proc_val = duration_feature(proc_val, val_tracking)
proc_all = duration_feature(proc_all, all_tracking)
proc_test = duration_feature(proc_test, test_tracking)

/anaconda/envs/kaggle/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [9]:
X_train = proc_train.drop('sid', axis=1).reset_index(drop=True)
y_train = trn_session.target.reset_index(drop=True)

X_val = proc_val.drop('sid', axis=1).reset_index(drop=True)
y_val = val_session.target.reset_index(drop=True)

X_test = proc_test.drop('sid', axis=1).reset_index(drop=True)

temp_columns = X_train.columns
X_val = X_val[temp_columns]
X_test = X_test[temp_columns]

In [12]:
from sklearn.decomposition import PCA

X_all = proc_all.drop('sid', axis=1).reset_index(drop=True)
y_all = all_session.target.reset_index(drop=True)

X_test = proc_test.drop('sid', axis=1).reset_index(drop=True)
temp_columns = X_all.columns
X_test = X_test[temp_columns]

scaler = StandardScaler()
scaler.fit(X_all)
X_all = scaler.transform(X_all)
X_test = scaler.transform(X_test)

pca = PCA(n_components = 10).fit(X_all)

X_all_pca = pd.DataFrame(pca.transform(X_all))
X_test_pca = pd.DataFrame(pca.transform(X_test))

X_all = pd.concat([pd.DataFrame(X_all, columns=temp_columns), pd.DataFrame(X_all_pca)], axis=1)
X_test = pd.concat([pd.DataFrame(X_test, columns=temp_columns), pd.DataFrame(X_test_pca)], axis=1)

In [13]:
X_all.head()

,ADD_TO_BASKET_CAROUSEL_type,ADD_TO_BASKET_LP_type,ADD_TO_BASKET_LR_type,ADD_TO_BASKET_PA_type,ADD_TO_BASKET_SHOW_CASE_type,CAROUSEL_type,LIST_PRODUCT_type,PA_type,PRODUCT_CAROUSEL_type,PRODUCT_LP_type,PRODUCT_LR_type,PRODUCT_PA_type,PRODUCT_SHOW_CASE_type,PURCHASE_PRODUCT_CAROUSEL_type,PURCHASE_PRODUCT_LP_type,PURCHASE_PRODUCT_LR_type,PURCHASE_PRODUCT_PA_type,PURCHASE_PRODUCT_SHOW_CASE_type,PURCHASE_PRODUCT_UNKNOW_ORIGIN_type,SEARCH_type,SHOW_CASE_type,SUM_TYPE_type,ADD_TO_BASKET_sim_type,CAROUSEL_sim_type,LIST_PRODUCT_sim_type,PA_sim_type,PRODUCT_sim_type,PURCHASE_PRODUCT_sim_type,SEARCH_sim_type,SHOW_CASE_sim_type,SUM_TYPE_SIMPLIFIED_sim_type,TOTAL_DURATION,0,1,2,3,4,5,6,7,8,9
0,-0.13765,-0.095176,-0.324503,-0.042814,-0.053234,-0.019736,-0.35745,-0.712154,-0.393928,-0.227696,1.317391,-0.170021,-0.180313,-0.031101,-0.017806,-0.065901,-0.011609,-0.01134,-0.07732,0.570503,-0.430156,-0.433643,-0.375153,-0.019736,-0.35745,-0.712154,0.715825,-0.099268,0.570503,-0.430156,-0.433643,0.307845,1.268369,0.942445,0.143585,-0.677491,-0.552715,-1.017447,-0.101435,0.360089,0.034325,-0.393044
1,-0.13765,-0.095176,-0.324503,-0.042814,-0.053234,0.196435,-0.35745,-0.712154,0.491024,-0.227696,1.162556,-0.170021,-0.180313,-0.031101,-0.017806,-0.065901,-0.011609,-0.01134,-0.07732,0.195186,-0.430156,0.197951,-0.375153,0.196435,-0.35745,-0.712154,1.117323,-0.099268,0.195186,-0.430156,0.197951,-0.288720,0.554129,1.522824,0.485947,-0.714966,-0.353063,-0.955266,0.377005,0.620819,-0.078899,-0.177089
2,-0.13765,-0.095176,-0.324503,-0.042814,-0.053234,-0.956479,-0.35745,0.895762,-0.393928,-0.227696,1.317391,-0.170021,-0.180313,-0.031101,-0.017806,-0.065901,-0.011609,-0.01134,-0.07732,0.570503,-0.430156,-0.433643,-0.375153,-0.956479,-0.35745,0.895762,0.715825,-0.099268,0.570503,-0.430156,-0.433643,-0.346940,1.875491,-0.506557,-0.851337,-0.484701,-0.878944,0.039262,0.351948,1.279285,-0.544646,-0.089294
3,-0.13765,-0.095176,-0.324503,-0.042814,-0.053234,-0.956479,-0.35745,-0.712154,-0.393928,-0.227696,-0.695464,-0.170021,-0.180313,-0.031101,-0.017806,-0.065901,-0.011609,-0.01134,-0.07732,2.196876,-0.430156,-0.433643,-0.375153,-0.956479,-0.35745,-0.712154,-1.023999,-0.099268,2.196876,-0.430156,-0.433643,-0.370200,3.114899,-0.385906,0.034111,-0.206055,-0.318827,-0.753769,-0.666346,-1.689476,0.639276,0.797476
4,-0.13765,-0.095176,-0.324503,-0.042814,-0.053234,-0.956479,-0.35745,-0.712154,-0.393928,-0.227696,-0.695464,-0.170021,-0.180313,-0.031101,-0.017806,-0.065901,-0.011609,-0.01134,-0.07732,2.196876,-0.430156,-0.503820,-0.375153,-0.956479,-0.35745,-0.712154,-1.023999,-0.099268,2.196876,-0.430156,-0.503820,-0.376343,3.110506,-0.404244,-0.010610,-0.223977,-0.359486,-0.780192,-0.729058,-1.681756,0.641185,0.800302


In [14]:
X_test.head()

,ADD_TO_BASKET_CAROUSEL_type,ADD_TO_BASKET_LP_type,ADD_TO_BASKET_LR_type,ADD_TO_BASKET_PA_type,ADD_TO_BASKET_SHOW_CASE_type,CAROUSEL_type,LIST_PRODUCT_type,PA_type,PRODUCT_CAROUSEL_type,PRODUCT_LP_type,PRODUCT_LR_type,PRODUCT_PA_type,PRODUCT_SHOW_CASE_type,PURCHASE_PRODUCT_CAROUSEL_type,PURCHASE_PRODUCT_LP_type,PURCHASE_PRODUCT_LR_type,PURCHASE_PRODUCT_PA_type,PURCHASE_PRODUCT_SHOW_CASE_type,PURCHASE_PRODUCT_UNKNOW_ORIGIN_type,SEARCH_type,SHOW_CASE_type,SUM_TYPE_type,ADD_TO_BASKET_sim_type,CAROUSEL_sim_type,LIST_PRODUCT_sim_type,PA_sim_type,PRODUCT_sim_type,PURCHASE_PRODUCT_sim_type,SEARCH_sim_type,SHOW_CASE_sim_type,SUM_TYPE_SIMPLIFIED_sim_type,TOTAL_DURATION,0,1,2,3,4,5,6,7,8,9
0,-0.13765,-0.095176,-0.324503,-0.042814,-0.053234,-0.956479,4.759748,-0.712154,-0.393928,-0.227696,-0.695464,-0.170021,-0.180313,-0.031101,-0.017806,-0.065901,-0.011609,-0.01134,-0.07732,-1.055871,-0.044044,0.408482,-0.375153,-0.956479,4.759748,-0.712154,-1.023999,-0.099268,-1.055871,-0.044044,0.408482,-0.332188,-1.863607,-3.334790,4.558398,0.725514,-2.093641,0.747529,-0.644031,-0.945395,0.182824,-0.029655
1,-0.13765,-0.095176,-0.324503,-0.042814,-0.053234,-0.956479,-0.357450,-0.712154,-0.393928,-0.227696,-0.695464,-0.170021,-0.180313,-0.031101,-0.017806,-0.065901,-0.011609,-0.01134,-0.07732,1.383690,1.114292,-0.433643,-0.375153,-0.956479,-0.357450,-0.712154,-1.023999,-0.099268,1.383690,1.114292,-0.433643,-0.359264,1.788232,-1.459383,-0.081537,-0.412905,0.963791,-1.360606,-0.694712,-1.346371,0.470497,0.675596
2,-0.13765,-0.095176,-0.324503,-0.042814,-0.053234,-0.956479,-0.357450,-0.712154,-0.393928,-0.227696,1.317391,-0.170021,-0.180313,-0.031101,-0.017806,-0.065901,-0.011609,-0.01134,-0.07732,1.383690,-0.430156,-0.433643,-0.375153,-0.956479,-0.357450,-0.712154,0.715825,-0.099268,1.383690,-0.430156,-0.433643,-0.324156,2.741238,0.303470,0.368993,-0.683189,-0.588497,-1.227168,-0.302332,0.539625,0.097025,0.097953
3,-0.13765,-0.095176,-0.324503,-0.042814,-0.053234,-0.956479,-0.357450,0.895762,-0.393928,-0.227696,1.317391,-0.170021,-0.180313,-0.031101,-0.017806,-0.065901,-0.011609,-0.01134,-0.07732,0.570503,-0.430156,-0.433643,-0.375153,-0.956479,-0.357450,0.895762,0.715825,-0.099268,0.570503,-0.430156,-0.433643,-0.275711,1.874483,-0.504576,-0.848111,-0.479286,-0.870833,0.050058,0.373993,1.274058,-0.547555,-0.096271
4,-0.13765,-0.095176,-0.324503,-0.042814,-0.053234,0.292512,-0.357450,1.431735,-0.393928,-0.227696,-0.695464,-0.170021,-0.180313,-0.031101,-0.017806,-0.065901,-0.011609,-0.01134,-0.07732,-1.055871,1.629109,-0.503820,-0.375153,0.292512,-0.357450,1.431735,-1.023999,-0.099268,-1.055871,1.629109,-0.503820,-0.348304,-1.765237,-2.085824,-2.128637,-0.288254,0.920147,0.257823,0.126131,-0.418819,-0.490389,-0.188278


In [15]:
y_all.head()

0    False
1    False
2    False
3    False
4    False
Name: target, dtype: bool

In [29]:
params = {
    'objective' :'binary',
    'max_depth' : 9,
    'num_leaves': 50,
    'min_child_samples': 10,
    'scale_pos_weight': 1.0,
    'subsample': 0.866999534037008,
    'colsample_bytree': 0.2,
    'boosting_type' : 'dart',
    'xgboost_dart_mode': True,
    'learning_rate': 0.0459,
    'lambda_l2': 5.0,
    #'metric': 'binary_logloss',
    #'bagging_fraction': 0.5,
}

test_session['target'] = test_lgbm(lgbm, params, X_all, y_all, X_test, test_size=0.2, random_seeds=[8, 15, 42, 60], cat_features=[])[0]

No. estimators: 231 | Train log loss: 0.260318082219809 | Val log loss: 0.26492404823708604
No. estimators: 270 | Train log loss: 0.2580610914863455 | Val log loss: 0.2693926375782864
No. estimators: 269 | Train log loss: 0.25731758135143257 | Val log loss: 0.2721332794385017
No. estimators: 285 | Train log loss: 0.2574732913524768 | Val log loss: 0.2702023305645445

Train logloss: 25.83 +/- 0.24 | Val logloss: 26.92 +/- 0.53



In [30]:
test_session.to_csv(f'data/submissions/chu_10_submit_bag02.csv', index=False)